In [ ]:
!pip install transformers
!pip install pysentimiento
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from pysentimiento.preprocessing import preprocess_tweet
from google.colab import files
from pysentimiento import create_analyzer
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
import tensorflow as tf
import pandas as pd
import random
import numpy as np

uploaded = files.upload()



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 29.8 MB/s 
     |████████████████████████████████| 182 kB 72.8 MB/s 
     |████████████████████████████████| 7.6 MB 74.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 53.7 MB/s 
     |████████████████████████████████| 175 kB 90.4 MB/s 
     |████████████████████████████████| 212 kB 93.6 MB/s 
     |████████████████████████████████| 132 kB 89.3 MB/s 
     |████████████████████████████████| 127 kB 96.2 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=3501dd3f85c74e1445dbe07b3e91ebb55a102c168a898e327fba2c7b69533c9d
  Stored in directory: /root/.cache/pip/wheels/5e/8c/80/c3646df8201ba6f5070297fe3779a4b70265d0bfd961c15302
Successfully built emoji
  Attempting uninstall: urllib3
    Found existing instal

Saving cleaned_tweets.csv to cleaned_tweets.csv
Saving full_articles.csv to full_articles.csv


In [ ]:
tweets = pd.read_csv('cleaned_tweets.csv')
tweets = tweets.rename(columns={'tweets': "data"})

articles = pd.read_csv('full_articles.csv')
articles = articles.rename(columns={'hechos': "data"})

def check_balance(df):
    print(df.label.value_counts())
    
print(check_balance(tweets))
print(check_balance(articles))

art_nolabel = articles.loc[articles['label'] == 3]
art_label = articles.loc[articles['label'] < 2]

0    5017
1    5012
Name: label, dtype: int64
None
3    7932
0      72
1      51
Name: label, dtype: int64
None


In [ ]:
X_train_t, X_test_t= train_test_split(tweets, test_size=0.20, random_state=42)
X_train_a, X_test_a= train_test_split(art_label, test_size=0.20, random_state=42)

Convert data to tensors

In [ ]:
def convert_data_to_examples(train, test): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x["data"], 
                                                          text_b = None,
                                                          label = x["label"]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x["data"], 
                                                          text_b = None,
                                                          label = x["label"]), axis = 1)

  return train_InputExamples, validation_InputExamples

  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )

InputExample(guid=None,
             text_a = "Hello, world",
             text_b = None,
             label = 1)


InputExample(guid=None, text_a='Hello, world', text_b=None, label=1)

# BERT base uncased

## test tweets predict tweets

In [ ]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model.summary()

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [ ]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(X_train_t, X_test_t)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data)



/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch 1/2
502/502 [==============================] - 447s 831ms/step - loss: 0.4804 - accuracy: 0.7612 - val_loss: 0.3998 - val_accuracy: 0.8270
Epoch 2/2
502/502 [==============================] - 416s 828ms/step - loss: 0.1999 - accuracy: 0.9227 - val_loss: 0.5404 - val_accuracy: 0.8185


In [ ]:
model.fit(train_data, epochs=3, validation_data=validation_data)

Epoch 1/3
502/502 [==============================] - 416s 828ms/step - loss: 0.0885 - accuracy: 0.9676 - val_loss: 0.6696 - val_accuracy: 0.8290
Epoch 2/3
502/502 [==============================] - 415s 827ms/step - loss: 0.0488 - accuracy: 0.9835 - val_loss: 0.6603 - val_accuracy: 0.8360
Epoch 3/3
502/502 [==============================] - 415s 826ms/step - loss: 0.0211 - accuracy: 0.9919 - val_loss: 0.8460 - val_accuracy: 0.8445


## test news predict news

In [ ]:
art_label = art_label.dropna(subset=['data', 'label'])
art_nolabel = art_nolabel.dropna(subset=['data', 'label'])

In [ ]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

model.summary()

train_InputExamples, validation_InputExamples = convert_data_to_examples(art_label, art_nolabel)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data)



All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_151 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch 1/2
8/8 [==============================] - 97s 11s/step - loss: 0.6774 - accuracy: 0.5805 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/2
8/8 [==============================] - 75s 11s/step - loss: 0.6171 - accuracy: 0.6525 - val_loss: nan - val_accuracy: 0.0000e+00


In [ ]:
model.fit(train_data, epochs=3, validation_data=validation_data)


Epoch 1/3
8/8 [==============================] - 88s 12s/step - loss: 0.5319 - accuracy: 0.7458 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/3
8/8 [==============================] - 76s 11s/step - loss: 0.3950 - accuracy: 0.8559 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 3/3
8/8 [==============================] - 76s 11s/step - loss: 0.3190 - accuracy: 0.9110 - val_loss: nan - val_accuracy: 0.0000e+00


## Train with tweets predict news


In [ ]:
prediction_text = list(art_nolabel["data"])
j = 0
predictions = []
for i in range (0,793):
  pred = prediction_text[j:(j+10)]
  tf_batch = tokenizer(pred, padding=True, truncation=True, return_tensors='tf')
  tf_outputs = model(tf_batch)
  tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
  labels = ['1','0']
  label = tf.argmax(tf_predictions, axis=1)
  label = label.numpy()
  pred_labels = [labels[label[i]] for i in range(len(pred))]
  predictions.append((pd.DataFrame(data=[pred,pred_labels]).T))
  j += 10

  

In [ ]:
len(predictions)
all_predictions = pd.concat(predictions)
all_predictions.to_csv("all_predictions.csv",index=False)
all_predictions.to_csv("all_predictions.txt",index=False)
from google.colab import files
files.download("all_predictions.txt")

In [ ]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

model.summary()

train_InputExamples, validation_InputExamples = convert_data_to_examples(X_train_t, art_nolabel)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data)
## missing process multiple epochs to choose best value

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_227 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch 1/2
502/502 [==============================] - 489s 935ms/step - loss: 0.4556 - accuracy: 0.7864 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/2
502/502 [==============================] - 467s 930ms/step - loss: 0.1998 - accuracy: 0.9217 - val_loss: nan - val_accuracy: 0.0000e+00


In [ ]:
prediction_text = list(art_nolabel["data"])
j = 0
predictions = []
for i in range (0,793):
  pred = prediction_text[j:(j+10)]
  tf_batch = tokenizer(pred, padding=True, truncation=True, return_tensors='tf')
  tf_outputs = model(tf_batch)
  tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
  labels = ['1','0']
  label = tf.argmax(tf_predictions, axis=1)
  label = label.numpy()
  pred_labels = [labels[label[i]] for i in range(len(pred))]
  predictions.append((pd.DataFrame(data=[pred,pred_labels]).T))
  j += 10
  

In [ ]:
len(predictions)
all_predictions = pd.concat(predictions)
all_predictions.to_csv("all_predictions.csv",index=False)
all_predictions.to_csv("all_predictions.txt",index=False)
from google.colab import files
files.download("all_predictions.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download("all_predictions.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# BERT multilingual base uncased

test tweets predict tweets

In [ ]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-multilingual-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")

model.summary()

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/999M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  167356416 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 167,357,954
Trainable params: 167,357,954
Non-trainable params: 0
_________________________________________________________________


In [ ]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(X_train_t, X_test_t)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data)


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
model.fit(train_data, epochs=3, validation_data=validation_data)

Epoch 1/3
502/502 [==============================] - 90s 130ms/step - loss: 0.3720 - accuracy: 0.8295 - val_loss: 0.3435 - val_accuracy: 0.8744
Epoch 2/3
502/502 [==============================] - 62s 124ms/step - loss: 0.1387 - accuracy: 0.9501 - val_loss: 0.4574 - val_accuracy: 0.8519
Epoch 3/3
502/502 [==============================] - 62s 124ms/step - loss: 0.0602 - accuracy: 0.9797 - val_loss: 0.5512 - val_accuracy: 0.8594


## test news predict news

In [ ]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-multilingual-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")

model.summary()

train_InputExamples, validation_InputExamples = convert_data_to_examples(art_label, art_nolabel)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  167356416 
                                                                 
 dropout_113 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 167,357,954
Trainable params: 167,357,954
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_data, epochs=2, validation_data=validation_data)

Epoch 1/2
8/8 [==============================] - 34s 2s/step - loss: 0.6780 - accuracy: 0.5847 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/2
8/8 [==============================] - 12s 2s/step - loss: 0.6106 - accuracy: 0.6780 - val_loss: nan - val_accuracy: 0.0000e+00


In [ ]:
model.fit(train_data, epochs=3, validation_data=validation_data)

Epoch 1/3
8/8 [==============================] - 36s 2s/step - loss: 0.6599 - accuracy: 0.5678 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/3
8/8 [==============================] - 13s 2s/step - loss: 0.5478 - accuracy: 0.7797 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 3/3
8/8 [==============================] - 13s 2s/step - loss: 0.3233 - accuracy: 0.9449 - val_loss: nan - val_accuracy: 0.0000e+00


In [ ]:
prediction_text = list(art_nolabel["data"])
j = 0
predictions = []
for i in range (0,793):
  pred = prediction_text[j:(j+10)]
  tf_batch = tokenizer(pred, padding=True, truncation=True, return_tensors='tf')
  tf_outputs = model(tf_batch)
  tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
  labels = ['1','0']
  label = tf.argmax(tf_predictions, axis=1)
  label = label.numpy()
  pred_labels = [labels[label[i]] for i in range(len(pred))]
  predictions.append((pd.DataFrame(data=[pred,pred_labels]).T))
  j += 10

  len(predictions)
all_predictions = pd.concat(predictions)
all_predictions.to_csv("bert_ml_tw.csv",index=False)
from google.colab import files
files.download("bert_ml_tw.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Train tweets predict news

In [ ]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-multilingual-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")

model.summary()

train_InputExamples, validation_InputExamples = convert_data_to_examples(X_train_t, art_nolabel)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  167356416 
                                                                 
 dropout_151 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 167,357,954
Trainable params: 167,357,954
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
502/502 [==============================] - 94s 147ms/step - loss: 0.3526 - accuracy: 0.8398 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/2
502/502 [==============================] - 71s 142ms/step - loss: 0.1337 - accuracy: 0.9520 - val_loss: nan - val_accuracy: 

Pysentimiento no training

In [ ]:
#pysentimiento
tokenizer = AutoTokenizer.from_pretrained("pysentimiento/robertuito-sentiment-analysis")
hate_speech_analyzer = create_analyzer(task="hate_speech", lang="es")

#predicting tweets with pysentimiento no train
art_nums = [random.randint(0, len(tweets["data"])-1) for a in range(5)]

for x in art_nums:
  print(tweets["data"][x])
  print(hate_speech_analyzer.predict(tweets["data"][x]))

#predicting news with pysentimiento no train  examples 
article='La Fiscalía lamenta la muerte de Mara Fernanda Castilla, indicó que a través de las investigaciones realizadas establecieron que la víctima fue privada de la vida por su agresor e indicaron que posiblemente fue asesinada en un motel de Puebla. El gobierno del estado de Puebla ofreció sus condolencias para que los culpables sean sancionados con los máximos que permita la ley. PUBLICIDAD Tras una denuncia que presentaron los padres de Mara Fernanda tras no ser localizada luego de haber tomado un servicio de Cabify, autoridades realizaron las diligencias necesarias. La Fiscalía indicó que el conductor se presentó a declarar, quien manifestó que Mara sí había tomado el transporte sin embargo indicó que pero que la joven había solicitado bajar de la unidad en un punto cercano a su domicilio sobre las misma calle. Una vez confrontada la declaración de Ricardo "N", autoridades establecieron inconsistencias en los hechos, por lo que ordenaron su detención por caso emergente, mismo que se realizó el martes 12 en Tlaxcala, en su recidencia. Tras el análisis de telefonía de Ricardo y Mara, autoridades indicaron que ambos estuvieron en los mismos lugares después la desaparición de esta última, incluyendo el domicilio de sospechoso en el estado de Tlaxcala. El sospechoso trasladó a la estudiante al motel de la 11 Sur y 105 Poniente, llamado "Motel del Sur", antes hizo una parada para comprar cigarros. Ingresó a la habitación 25 a las 6:47 de la mañana y salió a 8:15 horas. Lugar donde se reportó el faltante de una sábana y una toalla, elementos con los que fue encontrada envuelta la víctima. Al salir de motel, Ricardo tomó Periférico con dirección a la autopista México-Puebla, retornó en un punto y se dirigió a la junta auxiliar de Santa María Xonacatepec, para luego regresar a la ciudad a realizar otras actividades. Aunque Ricardo "N" se presentó el domingo 9 de septiembre a declarar ante el Ministerio Público sobre la desaparición de Mara, fue el rastreo a su celular lo que permitió revelar que él asesinó a la joven. PUBLICIDAD Al solicitar a la compañía telefónica Telcel, el registro de llamadas y datos de los celulares tanto de Ricardo como de Mara Fernanda, descubrieron que hubo actividad los días 8, 9 y 12 (cuando se le aprehendió al hoy inculpado) en el municipio de Santa Úrsula Zimatepec. A pesar de que los dos teléfonos estuvieron juntos desde el viernes 8 de septiembre, cuando Ricardo se presentó a declarar por voluntad propia, nunca mencionó que en su poder tenía el celular de la joven, acotó en determinado momento la Fiscalía. Este viernes, justo a siete días de su desaparición, el cadáver de Mara Fernanda fue hallado en un lote baldío alrededor de las 13 horas, por lo que el conductor de Cabify está siendo procesado por el delito de feminicidio. El titular de la Fiscalía General del Estado, Víctor Carrancá Bourguet, dijo que se le practicarán exámenes científicos al cuerpo de Mara Fernanda para determinar si hubo violación, así como las causas y determinar el día de su muerte, que hasta el momento son inciertas. Así desapareció Mara, estudiante de UPAEP que viajaba en Cabify 1.- Jueves 7 de septiembre Mara Fernanda acude por la tarde-noche con unos amigos al corredor de antros de la 14 Oriente en San Andrés Cholula. 2.- Alrededor de las 5 de la madrugada del viernes 8, pide el servicio de un taxi de la empresa privada Cabify. 3.- Minutos después llega por ella, afuera del antro “Bronx”, Ricardo Alexis, en un automóvil gris, marca Chevrolet, tipo Sonic, con placas UAY-(XXXX) de Puebla 4.- De acuerdo a la plataforma de la empresa de transporte privado, el recorrido hasta el lugar de destino fue de 44 minutos con 6 segundos, de Cholula al fraccionamiento Torres de Mayorazgo, al sur de la ciudad, sitio al que llegó a las 05:48 horas. 5.- Se sabe que las cámaras del fraccionamiento detectaron la llegada del taxi, a la hora mencionada, pero tras permanecer alrededor de media hora detenido, nunca se apreció que descendiera la joven. 6.- Al pasar las horas y no saber nada de la universitaria, sus familiares y conocidos comenzaron una intensa campaña de búsqueda en redes sociales; asimismo acudieron a la Fiscalía a presentar la denuncia correspondiente. 7.- El domingo, de acuerdo a la empresa Cabify, el chofer Ricardo Alexis acudió voluntariamente a la Fiscalía para declarar sobre lo que sabía. Se dice que manifestó haber escuchado que Mara se puso de acuerdo con alguien más para verse, pero no se dieron más detalles. 8.- El lunes por la tarde, la Fiscalía en su cuenta de twitter posteó un mensaje en el que indicaba que seguían las investigaciones. 9.- Ayer a las 13 horas, agentes estatales de investigación de Puebla, en coordinación con sus pares de la Procuraduría General de Justicia de Tlaxcala, aprehendieron en el municipio de Terrenate, a Ricardo Alexis, por su probable participación en el delito de privación ilegal de la libertad. 10.- El 15 de septiembre, un juez le dictó prisión preventiva a el conductor de Cabify. 11.- El 15 de septiembre a las 13 horas apareció el cadáver de la joven, tras siete días de intensa búsqueda.'
print(hate_speech_analyzer.predict(article))
article_title = 'Amigas que abandonaron a Debanhi dan su versión de los hechos… Vodka, discusiones y mordidas que desencadenaron en la muerte de la joven'
print(hate_speech_analyzer.predict(article_title))

prediction_text = list(art_nolabel["data"])
j = 0
predictions = []
for i in range (0,793):
  pred = prediction_text[j:(j+10)]
  tf_batch = tokenizer(pred, padding=True, truncation=True, return_tensors='tf')
  tf_outputs = model(tf_batch)
  tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
  labels = ['1','0']
  label = tf.argmax(tf_predictions, axis=1)
  label = label.numpy()
  pred_labels = [labels[label[i]] for i in range(len(pred))]
  predictions.append((pd.DataFrame(data=[pred,pred_labels]).T))
  j += 10
len(predictions)
all_predictions = pd.concat(predictions)
all_predictions.to_csv("all_predictions_pysentnotrain.csv",index=False)
all_predictions.to_csv("all_predictions_pysentnotrain.txt",index=False)
from google.colab import files
files.download("all_predictions_pysentnotrain.csv")
files.download("all_predictions_pysentnotrain.txt")

pysentimiento fine-tuned tweets test tweets

In [ ]:
model = TFBertForSequenceClassification.from_pretrained("pysentimiento/robertuito-sentiment-analysis")
tokenizer = AutoTokenizer.from_pretrained("pysentimiento/robertuito-sentiment-analysis")

train_InputExamples, validation_InputExamples = convert_data_to_examples(X_train_t, X_test_t)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])
model.fit(train_data, epochs=2, validation_data=validation_data)

https://huggingface.co/pysentimiento/robertuito-sentiment-analysis/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpyrtf0nvc


Downloading:   0%|          | 0.00/925 [00:00<?, ?B/s]

storing https://huggingface.co/pysentimiento/robertuito-sentiment-analysis/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/034fd09e9530137fb6e6c042529972a92619fb02df8b40e7a4cfc50090943c46.98e658c5b8878c67807e3287b07db9608dbd08a5b69ce09979c447190bbe8077
creating metadata file for /root/.cache/huggingface/transformers/034fd09e9530137fb6e6c042529972a92619fb02df8b40e7a4cfc50090943c46.98e658c5b8878c67807e3287b07db9608dbd08a5b69ce09979c447190bbe8077
loading configuration file https://huggingface.co/pysentimiento/robertuito-sentiment-analysis/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/034fd09e9530137fb6e6c042529972a92619fb02df8b40e7a4cfc50090943c46.98e658c5b8878c67807e3287b07db9608dbd08a5b69ce09979c447190bbe8077
You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Model config BertConfig {
  "_name_or_path": "pysentimiento/robertuito-b

Downloading:   0%|          | 0.00/415M [00:00<?, ?B/s]

storing https://huggingface.co/pysentimiento/robertuito-sentiment-analysis/resolve/main/tf_model.h5 in cache at /root/.cache/huggingface/transformers/01a50cf9cfc868f0b2d98c1dd5b6396a2c90f166bd352c4a6641d914ebbc71b1.5dc5a1fd43233c80705ca844b9a557d590cc7635255ed60480eae82f1848885b.h5
creating metadata file for /root/.cache/huggingface/transformers/01a50cf9cfc868f0b2d98c1dd5b6396a2c90f166bd352c4a6641d914ebbc71b1.5dc5a1fd43233c80705ca844b9a557d590cc7635255ed60480eae82f1848885b.h5
loading weights file https://huggingface.co/pysentimiento/robertuito-sentiment-analysis/resolve/main/tf_model.h5 from cache at /root/.cache/huggingface/transformers/01a50cf9cfc868f0b2d98c1dd5b6396a2c90f166bd352c4a6641d914ebbc71b1.5dc5a1fd43233c80705ca844b9a557d590cc7635255ed60480eae82f1848885b.h5
Some layers from the model checkpoint at pysentimiento/robertuito-sentiment-analysis were not used when initializing TFBertForSequenceClassification: ['roberta', 'classifier/out_proj/kernel:0', 'classifier/dense/bias:0', 

Epoch 1/2
502/502 [==============================] - 90s 128ms/step - loss: 0.5212 - accuracy: 0.7313 - val_loss: 0.4927 - val_accuracy: 0.7986
Epoch 2/2
502/502 [==============================] - 61s 122ms/step - loss: 0.2434 - accuracy: 0.9088 - val_loss: 0.5107 - val_accuracy: 0.8066


In [ ]:
model.fit(train_data, epochs=3, validation_data=validation_data)

Epoch 1/3
502/502 [==============================] - 61s 121ms/step - loss: 0.1352 - accuracy: 0.9542 - val_loss: 0.5795 - val_accuracy: 0.8220
Epoch 2/3
502/502 [==============================] - 61s 121ms/step - loss: 0.0773 - accuracy: 0.9739 - val_loss: 0.7247 - val_accuracy: 0.8195
Epoch 3/3
502/502 [==============================] - 61s 121ms/step - loss: 0.0449 - accuracy: 0.9859 - val_loss: 0.9603 - val_accuracy: 0.8166


pysentimiento fine-tuned articles test articles

In [ ]:
model = TFBertForSequenceClassification.from_pretrained("pysentimiento/robertuito-sentiment-analysis")
tokenizer = AutoTokenizer.from_pretrained("pysentimiento/robertuito-sentiment-analysis")


Downloading:   0%|          | 0.00/925 [00:00<?, ?B/s]

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Downloading:   0%|          | 0.00/415M [00:00<?, ?B/s]

Some layers from the model checkpoint at pysentimiento/robertuito-sentiment-analysis were not used when initializing TFBertForSequenceClassification: ['roberta', 'classifier/dense/kernel:0', 'classifier/out_proj/kernel:0', 'classifier/dense/bias:0', 'classifier/out_proj/bias:0']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at pysentimiento/robertuito-sentiment-analysis and are newly initialized: ['bert', 'classifier/kernel:0', 'classifier/bias:0']
Yo

Downloading:   0%|          | 0.00/334 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/838k [00:00<?, ?B/s]

In [ ]:
art_label = art_label.dropna(subset=['data', 'label'])
art_nolabel = art_nolabel.dropna(subset=['data', 'label'])

train_InputExamples, validation_InputExamples = convert_data_to_examples(art_label, art_nolabel)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])
model.fit(train_data, epochs=2, validation_data=validation_data)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch 1/2
8/8 [==============================] - 41s 2s/step - loss: 0.7821 - accuracy: 0.5466 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/2
8/8 [==============================] - 12s 2s/step - loss: 0.7045 - accuracy: 0.5381 - val_loss: nan - val_accuracy: 0.0000e+00


In [ ]:
prediction_text = list(art_nolabel["data"])
j = 0
predictions = []
for i in range (0,793):
  pred = prediction_text[j:(j+10)]
  tf_batch = tokenizer(pred, padding=True, truncation=True, return_tensors='tf')
  tf_outputs = model(tf_batch)
  tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
  labels = ['1','0']
  label = tf.argmax(tf_predictions, axis=1)
  label = label.numpy()
  pred_labels = [labels[label[i]] for i in range(len(pred))]
  predictions.append((pd.DataFrame(data=[pred,pred_labels]).T))
  j += 10

  len(predictions)
all_predictions = pd.concat(predictions)
all_predictions.to_csv("all_predictions.txt",index=False)
from google.colab import files
files.download("all_predictions.txt")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
model.fit(train_data, epochs=3, validation_data=validation_data)

Epoch 1/3
8/8 [==============================] - 12s 2s/step - loss: 0.6832 - accuracy: 0.5975 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/3
8/8 [==============================] - 12s 2s/step - loss: 0.6950 - accuracy: 0.5636 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 3/3
8/8 [==============================] - 12s 2s/step - loss: 0.6864 - accuracy: 0.5805 - val_loss: nan - val_accuracy: 0.0000e+00


pysentimiento fine-tuned tweets test articles

In [ ]:
model = TFBertForSequenceClassification.from_pretrained("pysentimiento/robertuito-sentiment-analysis")
tokenizer = AutoTokenizer.from_pretrained("pysentimiento/robertuito-sentiment-analysis")

train_InputExamples, validation_InputExamples = convert_data_to_examples(X_train_t, art_nolabel)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])
model.fit(train_data, epochs=2, validation_data=validation_data)


You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some layers from the model checkpoint at pysentimiento/robertuito-sentiment-analysis were not used when initializing TFBertForSequenceClassification: ['roberta', 'classifier/dense/kernel:0', 'classifier/out_proj/kernel:0', 'classifier/dense/bias:0', 'classifier/out_proj/bias:0']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForSequenceClassification were not initialized fro

Epoch 1/2
502/502 [==============================] - 92s 143ms/step - loss: 0.5141 - accuracy: 0.7399 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/2
502/502 [==============================] - 68s 136ms/step - loss: 0.2453 - accuracy: 0.9016 - val_loss: nan - val_accuracy: 0.0000e+00


In [ ]:
prediction_text = list(art_nolabel["data"])
j = 0
predictions = []
for i in range (0,793):
  pred = prediction_text[j:(j+10)]
  tf_batch = tokenizer(pred, padding=True, truncation=True, return_tensors='tf')
  tf_outputs = model(tf_batch)
  tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
  labels = ['1','0']
  label = tf.argmax(tf_predictions, axis=1)
  label = label.numpy()
  pred_labels = [labels[label[i]] for i in range(len(pred))]
  predictions.append((pd.DataFrame(data=[pred,pred_labels]).T))
  j += 10

  len(predictions)
all_predictions = pd.concat(predictions)
all_predictions.to_csv("py_tw_art.txt",index=False)
from google.colab import files
files.download("py_tw_art.txt")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>